# o'reillyのネゴザメ言語モデルの本
RNN Search（Attention）

## [目次](TableOfContents.ipynb)
- [環境準備](#環境準備)
  - [インストール](#インストール)
  - [インポート](#インポート)
- [RNN Search（Attention）](#RNN_Search（Attention）)
  - [Attention](#Attention)
    - [Weight Sumレイヤ](#Weight_Sumレイヤ)
    - [Attention Weightレイヤ](#Attention_Weightレイヤ)
    - [Attentionレイヤ](#Attentionレイヤ)
    - [Time Attentionレイヤ](#Time_Attentionレイヤ)
  - [RNN Search（Attention）](#RNN_Search（Attention）)
    - [Encoder](#Encoder)
    - [Decoder](#Decoder)
    - [Attention付きseq2seq](#Attention付きseq2seq)
    
## 参考
- https://github.com/oreilly-japan/deep-learning-from-scratch-2/tree/master/ch08
- [RNN Encoder-Decoder（Sequence-to-Sequence） - 開発基盤部会 Wiki](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?RNN%20Encoder-Decoder%EF%BC%88Sequence-to-Sequence%EF%BC%89)

## 環境準備

### インストール

In [ ]:
!pip install numpy
!pip install matplotlib

### インポート

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定

## RNN Search（Attention）

### Attention

#### Weight_Sumレイヤ

In [ ]:
# データとパラメータの形状に関する値を指定
N = 3 # バッチサイズ(入力する文章数)
T = 4 # Encoderの時系列サイズ(入力する単語数)
H = 5 # 隠れ状態のサイズ(LSTMレイヤの中間層のニューロン数)

##### 実装

In [ ]:
class _WeightSum:
    # 初期化メソッド
    def __init__(self):
        # 他のレイヤと対応させるための空のリストを作成
        self.params = [] # パラメータ
        self.grads = []  # 勾配
        
        # 中間変数の受け皿を初期化
        self.cache = None
    
    # 順伝播メソッド
    def forward(self, hs, a):
        # 変数の形状に関する値を取得
        N, T, H = hs.shape
        
        # Encoderの隠れ状態と同じ形状に複製
        ar = a.reshape(N, T, 1).repeat(H, axis=2)
        
        # コンテキスト(重み付き和)を計算
        t = hs * ar
        c = np.sum(t, axis=1)
        
        # 逆伝播の計算用に変数を保存
        self.cache = (hs, ar)
        return c
    
    # 逆伝播メソッド
    def backward(self, dc):
        # 変数を取得
        hs, ar = self.cache
        
        # 変数の形状に関する値を取得
        N, T, H = hs.shape
        
        # Sumノードの逆伝播を計算
        dt = dc.reshape(N, 1, H).repeat(T, axis=1)
        
        # 乗算ノードの逆伝播を計算
        dar = dt * hs
        dhs = dt * ar # Encoderの隠れ状態の勾配
        
        # Repeatノードの逆伝播を計算
        da = np.sum(dar, axis=2) # Attentionの重みの勾配
        return dhs, da

##### 実行

In [ ]:
# Weight Sumレイヤのインスタンスを作成
weightsum_layer = _WeightSum()

##### 順伝播

In [ ]:
# (簡易的に)Encoderの隠れ状態を作成
hs = np.random.randn(N, T, H)
print(np.round(hs, 2))
print(hs.shape)

# (簡易的に)Attentionの重みを作成
a = np.random.rand(N, T)
a /= np.sum(a, axis=1, keepdims=True) # 正規化
print(np.round(a, 2))
print(np.sum(a, axis=1))
print(a.shape)

# 順伝播を計算
c = weightsum_layer.forward(hs, a)
print(np.round(c, 2))
print(c.shape)

##### 逆伝播

In [ ]:
# (簡易的に)コンテキストの勾配を作成
dc = np.ones((N, H))
print(dc.shape)

# 逆伝播を計算
dhs, da = weightsum_layer.backward(dc)
print(np.round(dhs, 2))
print(dhs.shape)
print(np.round(da, 2))
print(da.shape)

#### Attention_Weightレイヤ

In [ ]:
# 実装済みクラスを読み込み
from nekozame.common.layers import Softmax

In [ ]:
# データとパラメータの形状に関する値を指定
N = 3 # バッチサイズ(入力する文章数)
T = 4 # Encoderの時系列サイズ(入力する単語数)
H = 5 # 隠れ状態のサイズ(LSTMレイヤの中間層のニューロン数)

##### 実装

In [ ]:
# Attention Weightレイヤの実装
class _AttentionWeight:
    # 初期化メソッド
    def __init__(self):
        # 他のレイヤと対応させるための空のリストを作成
        self.params = [] # パラメータ
        self.grads = []  # 勾配
        
        # Softmaxレイヤのインスタンスを作成
        self.softmax = Softmax()
        
        # 中間変数の受け皿を初期化
        self.cache = None
    
    # 順伝播メソッド
    def forward(self, hs, h):
        # 変数の形状に関する値を取得
        N, T, H = hs.shape
        
        # Encoderの隠れ状態同じ形状に複製
        hr = h.reshape((N, 1, H)).repeat(T, axis=1)
        
        # スコア(内積)を計算
        t = hs * hr
        s = np.sum(t, axis=2)
        
        # Attentionの重みに変換(正規化)
        a = self.softmax.forward(s)
        
        # 逆伝播の計算用に変数を保存
        self.cache = (hs, hr)
        return a
    
    # 逆伝播メソッド
    def backward(self, da):
        # 変数を取得
        hs, hr = self.cache
        
        # 形状に関する値を取得
        N, T, H = hs.shape
        
        # Softmaxレイヤの逆伝播(スコアの勾配)を計算
        ds = self.softmax.backward(da)
        
        # Sumノードの逆伝播を計算
        dt = ds.reshape((N, T, 1)).repeat(H, axis=2)
        
        # 乗算ノードの逆伝播を計算
        dhs = dt * hr # EncoderのT個の隠れ状態の勾配
        dhr = dt * hs
        
        # Repeatノードの逆伝播を計算
        dh = np.sum(dhr, axis=1) # Decoderのt番目の隠れ状態の勾配
        return dhs, dh

##### 実行

In [ ]:
# インスタンスを作成
attention_weight_layer = _AttentionWeight()

##### 順伝播

In [ ]:
# (簡易的に)EncoderのT個の隠れ状態を作成
hs = np.random.randn(N, T, H)
print(hs.shape)

# (簡易的に)Decoderの隠れ状態を作成
h = np.random.randn(N, H)
print(h.shape)

# 順伝播を計算
a = attention_weight_layer.forward(hs, h)
print(np.round(a, 2))
print(np.sum(a, axis=1))
print(a.shape)

##### 逆伝播

In [ ]:
# (簡易的に)逆伝播の入力を作成
da = np.random.randn(N, T)
print(da.shape)

# 逆伝播を計算
dhs, dh = attention_weight_layer.backward(da)
print(np.round(dhs, 3))
print(dhs.shape)
print(np.round(dh, 3))
print(dh.shape)

#### Attentionレイヤ

In [ ]:
# データとパラメータの形状に関する値を指定
N = 3 # バッチサイズ(入力する文章数)
T = 4 # Encoderの時系列サイズ(入力する単語数)
H = 5 # 隠れ状態のサイズ(LSTMレイヤの中間層のニューロン数)

##### 実装

In [ ]:
# Attentionレイヤの実装
class _Attention:
    # 初期化メソッド
    def __init__(self):
        # 他のレイヤと対応させるための空のリストを作成
        self.params = [] # パラメータ
        self.grads = []  # 勾配
        
        # レイヤのインスタンスを作成
        self.attention_weight_layer = _AttentionWeight()
        self.weight_sum_layer = _WeightSum()
        
        # Attentionの重みを初期化
        self.attention_weight = None
    
    # 順伝播メソッド
    def forward(self, hs, h):
        # Attentionの重みを計算
        a = self.attention_weight_layer.forward(hs, h)
        
        # EncoderのT個の隠れ状態の重み付け和を計算
        out = self.weight_sum_layer.forward(hs, a)
        
        # Attentionの重みを保存
        self.attention_weight = a
        return out
    
    # 逆伝播メソッド
    def backward(self, dout):
        # 各レイヤの逆伝播を計算
        dhs0, da = self.weight_sum_layer.backward(dout)
        dhs1, dh = self.attention_weight_layer.backward(da)
        
        # EncoderのT個の隠れ状態の勾配を計算
        dhs = dhs0 + dhs1
        
        # EncoderのT個の隠れ状態の勾配とDecoderのt番目の隠れ状態の勾配を出力
        return dhs, dh

##### 実行

In [ ]:
# インスタンスを作成
attention_layer = _Attention()

##### 順伝播

In [ ]:
# (簡易的に)EncoderのT個の隠れ状態を作成
hs = np.random.randn(N, T, H)
print(np.round(hs, 2))
print(hs.shape)

# (簡易的に)Decoderのt番目の隠れ状態を作成
h = np.random.randn(N, H)
print(np.round(h, 2))
print(h.shape)

# 順伝播を計算
c = attention_layer.forward(hs, h)
print(np.round(c, 2))
print(c.shape)

##### 逆伝播

In [ ]:
# (簡易的に)逆伝播の入力を作成
dc = np.ones((N, H))
print(dc.shape)

# 逆伝播を計算
dhs, dh = attention_layer.backward(dc)
print(np.round(dhs, 2))
print(dhs.shape)
print(np.round(dh, 2))
print(dh.shape)

#### Time_Attentionレイヤ

In [ ]:
# 変数の形状に関する値を指定
N = 3 # バッチサイズ(入力する文章数)
T_enc = 4 # Encoderの時系列サイズ(入力する単語数)
T_dec = 7 # Decoderの時系列サイズ(入力する単語数)
H = 5 # 隠れ状態のサイズ(LSTMレイヤの中間層のニューロン数)

##### 実装

In [ ]:
# Time Attentionレイヤの実装
class _TimeAttention:
    # 初期化メソッド
    def __init__(self):
        # 他のレイヤに対応するための空のリストを作成
        self.params = [] # パラメータ
        self.grads = []  # 勾配
        
        # Attentionの重みの受け皿を初期化
        self.attention_weights = None
    
    # 順伝播メソッド
    def forward(self, hs_enc, hs_dec):
        # 変数の形状に関する値を取得
        N, T, H = hs_dec.shape
        
        # T個のコンテキストの受け皿を初期化
        out = np.empty_like(hs_dec)
        
        # 受け皿を初期化
        self.layers = []
        self.attention_weights = []
        
        # Attentionレイヤごとに処理
        for t in range(T):
            # t番目のAttentionレイヤを作成
            layer = _Attention()
            
            # t番目のコンテキスト(隠れ状態の重み付き和)を計算
            out[:, t, :] = layer.forward(hs_enc, hs_dec[:, t, :])
            
            # t番目のレイヤとAttentionの重みを格納
            self.layers.append(layer)
            self.attention_weights.append(layer.attention_weight)
        
        return out
    
    # 逆伝播メソッド
    def backward(self, dout):
        # 変数の形状に関する値を取得
        N, T, H = dout.shape
        
        # EncoderとDecoderの隠れ状態の勾配を初期化
        dhs_enc = 0
        dhs_dec = np.empty_like(dout)
        
        # Attentionレイヤごとに処理
        for t in range(T):
            # t番目のAttentionレイヤを取得
            layer = self.layers[t]
            
            # EncoderとDecoderの隠れ状態の勾配を計算
            dhs, dh = layer.backward(dout[:, t, :])
            
            # EncoderのT個の隠れ状態の勾配を加算
            dhs_enc += dhs
            
            # Decoderのt番目の隠れ状態の勾配を格納
            dhs_dec[:, t, :] = dh
        
        return dhs_enc, dhs_dec

##### 実行

In [ ]:
# インスタンスを作成
time_attention_layer = _TimeAttention()

###### 順伝播

In [ ]:
# (簡易的に)EncoderのT個の隠れ状態を作成
hs_enc = np.random.randn(N, T_enc, H)
print(hs_enc.shape)

# (簡易的に)Decoderの隠れ状態を作成
hs_enc = np.random.randn(N, T_dec, H)
print(hs_enc.shape)

# 順伝播
cs = time_attention_layer.forward(hs_enc, hs_enc)
print(np.round(cs, 2))
print(cs.shape)

In [ ]:
# Attentionの重みを取得
attention_weights = time_attention_layer.attention_weights
print(np.round(attention_weights[0], 2)) # 0番目のAttentionの重み
print(np.sum(attention_weights[0], axis=1))
print(np.array(attention_weights[0]).shape)

###### 逆伝播

In [ ]:
# (簡易的に)逆伝播の入力を作成
dcs = np.random.randn(N, T_dec, H)
print(dcs.shape)

# 逆伝播を計算
dhs_enc, dhs_dec = time_attention_layer.backward(dcs)
print(dhs_enc.shape)
print(dhs_dec.shape)

### RNN_Search（Attention）

In [ ]:
# データとパラメータの形状に関する値を指定
N = 3 # バッチサイズ(入力する文章数)
V = 10 # 単語の種類数
D = 5 # 単語ベクトルの次元数(Embedレイヤの中間層のニューロン数)
H = 7 # 隠れ状態のサイズ(LSTMレイヤの中間層のニューロン数)
T_enc = 6 # Encoderの時系列サイズ(入力する単語数)
T_dec = 4 # Decoderの時系列サイズ(入力・予測する単語数)

#### Encoder

In [ ]:
# 実装済みのレイヤを読み込み
from nekozame.common.time_layers import TimeEmbedding
from nekozame.common.time_layers import TimeLSTM

##### 実装

In [ ]:
# Attention用のEncoderの実装
class _AttentionEncoder:
    # 初期化メソッド
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        # 変数の形状に関する値を取得
        V, D, H = vocab_size, wordvec_size, hidden_size
        
        # パラメータを初期化
        embed_W = (np.random.randn(V, D) * 0.01).astype('f')
        lstm_Wx = (np.random.randn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (np.random.randn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        
        # レイヤのインスタンスを作成
        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=False)
        
        # パラメータと勾配をリストに格納
        self.params = self.embed.params + self.lstm.params # パラメータ
        self.grads = self.embed.grads + self.lstm.grads    # 勾配
        
        # LSTMレイヤの中間変数を初期化
        self.hs = None
    
    # 順伝播メソッド
    def forward(self, xs):
        # 各レイヤの順伝播を計算
        xs = self.embed.forward(xs)
        hs = self.lstm.forward(xs)
        return hs
    
    # 逆伝播メソッド
    def backward(self, dhs):
        # 各レイヤの逆伝播を逆順に計算
        dout = self.lstm.backward(dhs)
        dout = self.embed.backward(dout)
        return dout

##### 実行

In [ ]:
# Encoderのインスタンスを作成
encoder = _AttentionEncoder(V, D, H)

###### 順伝播

In [ ]:
# (簡易的に)Encoderの入力データを作成
xs  = np.random.randint(low=0, high=V, size=(N, T_enc))
print(xs)
print(xs.shape)

# Encoderの隠れ状態を計算
hs_enc = encoder.forward(xs)
print(np.round(hs_enc, 3))
print(hs_enc.shape)

###### 逆伝播

In [ ]:
# (簡易的に)Encoderの隠れ状態の勾配を作成
dhs = np.random.randn(N, T_enc, H)
print(dhs.shape)

# 逆伝播を計算
dout = encoder.backward(dhs)
print(dout)

#### Decoder

In [ ]:
# 実装済みのレイヤを読み込み
from nekozame.common.time_layers import TimeAffine
from nekozame.common.time_layers import TimeEmbedding
from nekozame.common.time_layers import TimeLSTM

##### 実装

In [ ]:
# Attention付きDecoderの実装
class _AttentionDecoder:
    # 初期化メソッド
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        # 変数の形状に関する値を取得
        V, D, H = vocab_size, wordvec_size, hidden_size
        
        # パラメータを初期化
        embed_W = (np.random.randn(V, D) * 0.01).astype('f')
        lstm_Wx = (np.random.randn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (np.random.randn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (np.random.randn(2 * H, V) / np.sqrt(2 * H)).astype('f')
        affine_b = np.zeros(V).astype('f')
        
        # レイヤのインスタンスを作成
        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
        self.attention = _TimeAttention()
        self.affine = TimeAffine(affine_W, affine_b)
        
        # レイヤをリストに格納
        layers = [self.embed, self.lstm, self.attention, self.affine]
        
        # パラメータと勾配をリストに格納
        self.params = [] # パラメータ
        self.grads = []  # 勾配
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
        
    # 順伝播メソッド
    def forward(self, xs, enc_hs):
        # EncoderのT-1番目の隠れ状態を0番目のLSTMレイヤに入力
        h = enc_hs[:, -1]
        self.lstm.set_state(h)
        
        # 各レイヤの順伝播を計算
        out = self.embed.forward(xs)               # 単語ベクトル
        dec_hs = self.lstm.forward(out)            # 隠れ状態
        c = self.attention.forward(enc_hs, dec_hs) # コンテキスト
        out = np.concatenate((c, dec_hs), axis=2)  # コンテキストと隠れ状態を結合
        score = self.affine.forward(out)           # スコア
        return score
    
    # 逆伝播メソッド
    def backward(self, dscore):
        # Time Affineレイヤの逆伝播を計算
        dout = self.affine.backward(dscore)
        
        # 変数の形状に関する値を取得
        N, T, H2 = dout.shape
        H = H2 // 2
        
        # コンテキストの勾配と隠れ状態の勾配に分割
        dc, ddec_hs0 = dout[:, :, :H], dout[:, :, H:]
        
        # Time Attentionレイヤの逆伝播を計算
        denc_hs, ddec_hs1 = self.attention.backward(dc)
        
        # 分岐したDecoderの隠れ状態を合算
        ddec_hs = ddec_hs0 + ddec_hs1
        
        # Time LSTMレイヤの逆伝播を計算
        dout = self.lstm.backward(ddec_hs) # 単語ベクトルの勾配
        
        # 分岐したEncoderのT-1番目の隠れ状態の勾配を合算
        dh = self.lstm.dh
        denc_hs[:, -1] += dh
        
        # Time Embedレイヤの逆伝播を計算
        self.embed.backward(dout) # 出力はNone
        
        return denc_hs
    
    # 文章生成メソッド
    def generate(self, enc_hs, start_id, sample_size):
        # 文字IDの受け皿を初期化
        sampled = []
        
        # 区切り文字のIDを設定
        sample_id = start_id
        
        # Encoderの最後の隠れ状態をDecoderの最初のLSTMレイヤに入力
        h = enc_hs[:, -1, :]
        self.lstm.set_state(h)
        
        # 文章を生成
        for _ in range(sample_size):
            # 入力用に2次元配列に変換
            x = np.array(sample_id).reshape((1, 1))
            
            # スコアを計算
            out = self.embed.forward(x)                # 単語ベクトル
            dec_hs = self.lstm.forward(out)            # 隠れ状態
            c = self.attention.forward(enc_hs, dec_hs) # コンテキスト
            out = np.concatenate((c, dec_hs), axis=2)  # コンテキストと隠れ状態を結合
            score = self.affine.forward(out)           # スコア
            
            # スコアが最大の単語IDを取得
            sample_id = np.argmax(score.flatten()) # 入力単語を更新
            sampled.append(int(sample_id)) # サンプリングした単語を保存
        
        return sampled

##### 実行

In [ ]:
# Decoderのインスタンスを作成
decoder = _AttentionDecoder(V, D, H)

###### 逆伝播

In [ ]:
# (簡易的に)Decoderの入力データを作成
xs = np.random.randint(low=0, high=V, size=(N, T_dec))
print(xs)
print(xs.shape)

# (簡易的に)Encoderの隠れ状態を作成
hs_enc = np.random.randn(N, T_enc, H)
print(hs_enc.shape)

# スコアを計算
score = decoder.forward(xs, hs_enc)
print(score.shape)

###### 逆伝播

In [ ]:
# (簡易的に)スコアの勾配を作成
dscore = np.random.randn(N, T_dec, V)
print(dscore.shape)

# Encoderの隠れ状態の勾配を計算
dhs_enc = decoder.backward(dscore)
print(dhs_enc.shape)

### Attention付きseq2seq

In [ ]:
# 実装済みのレイヤを読み込み
from nekozame.common.time_layers import TimeSoftmaxWithLoss

#### 実装

In [ ]:
# Attention付きseq2seqの実装
class _AttentionSeq2seq:
    # 初期化メソッド
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        # 変数の形状に関する値を取得
        V, D, H = vocab_size, wordvec_size, hidden_size
        
        # 各レイヤのインスタンスを作成
        self.encoder = _AttentionEncoder(V, D, H)
        self.decoder = _AttentionDecoder(V, D, H)
        self.softmax = TimeSoftmaxWithLoss()
        
        # パラメータと勾配をリストに格納
        self.params = self.encoder.params + self.decoder.params # パラメータ
        self.grads = self.encoder.grads + self.decoder.grads    # 勾配
    
    # 順伝播メソッド
    def forward(self, xs, ts):
        # Decoder用の入植データを作成
        decoder_xs = ts[:, :-1] # 入力データ:(最後を除く)
        decoder_ts = ts[:, 1:]  # 教師データ:(最初を除く)
        
        # 各レイヤの順伝播を計算
        hs = self.encoder.forward(xs)
        score = self.decoder.forward(decoder_xs, hs)
        loss = self.softmax.forward(score, decoder_ts)
        return loss
    
    # 逆伝播メソッド
    def backward(self, dout=1):
        # 各レイヤの逆伝播を逆順に計算
        dout = self.softmax.backward(dout) # スコアの勾配
        dhs = self.decoder.backward(dout) # Encoderの隠れ状態の勾配
        dout = self.encoder.backward(dhs) # 出力はNone
        return dout
    
    # 文章生成メソッド
    def generate(self, xs, start_id, sample_size):
        # 問題文をエンコード
        hs = self.encoder.forward(xs)
        
        # 解答を生成
        sampled = self.decoder.generate(hs, start_id, sample_size)
        return sampled

#### 実行

In [ ]:
# seq2seqのインスタンスを作成
model = _AttentionSeq2seq(V, D, H)

##### 順伝播

In [ ]:
# (簡易的に)入力データを作成
xs = np.random.randint(low=0, high=V, size=(N, T_enc))
print(xs)
print(xs.shape)

# (簡易的に)教師データを作成
ts = np.random.randint(low=0, high=V, size=(N, T_dec + 1))
print(ts)
print(ts.shape)

# 順伝播を計算
loss = model.forward(xs, ts)
print(loss)

##### 逆伝播

In [ ]:
# 逆伝播を計算
dout = model.backward(dout=1)
print(dout)

#### 学習

In [ ]:
# 実装済みのクラスを読み込み
from nekozame.common.optimizer import Adam
from nekozame.common.trainer import Trainer

##### データの準備

###### データセットの読み込み

In [ ]:
# データセット読み込み用のモジュール
from nekozame.dataset import sequence

#　データセットの読み込み
(x_train, t_train), (x_test, t_test) = sequence.load_data('date.txt')
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

###### ディクショナリ

In [ ]:
# 文字と文字IDの変換用ディクショナリ変数の読み込み
char_to_id, id_to_char = sequence.get_vocab()
print(len(char_to_id))

# 文字と文字IDの変換用ディクショナリ変数を確認
print(char_to_id)
print(id_to_char)

###### レコードの確認

In [ ]:
# 表示するデータ番号を指定
n = 0

# 文字IDのリストを表示
print(x_train[n])
print(t_train[n])

# テキストに変換して表示
print(''.join([id_to_char[c_id] for c_id in x_train[n]]))
print(''.join([id_to_char[c_id] for c_id in t_train[n]]))

In [ ]:
# 表示するデータ番号を指定
n = 5

# 文字IDのリストを表示
print(x_train[n])
print(t_train[n])

# テキストに変換して表示
print(''.join([id_to_char[c_id] for c_id in x_train[n]]))
print(''.join([id_to_char[c_id] for c_id in t_train[n]]))

###### 入力文を反転

In [ ]:
reverse_x_train = x_train[:, ::-1]
reverse_x_test = x_test[:, ::-1]

##### 比較対象の再定義

In [ ]:
from nekozame.common.time_layers import TimeEmbedding
from nekozame.common.time_layers import TimeLSTM
from nekozame.common.time_layers import TimeAffine
from nekozame.common.time_layers import TimeSoftmaxWithLoss

###### Encoder

In [ ]:
# エンコーダーの実装
class Encoder:
    # 初期化メソッド
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        # 変数の形状に関する値を取得
        V, D, H = vocab_size, wordvec_size, hidden_size
        
        # パラメータを初期化
        embed_W = (np.random.randn(V, D) * 0.01).astype('f')
        lstm_Wx = (np.random.randn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (np.random.randn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        
        # レイヤのインスタンスを作成
        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=False)
        
        # パラメータと勾配をリストに格納
        self.params = self.embed.params + self.lstm.params # パラメータ
        self.grads = self.embed.grads + self.lstm.grads    # 勾配
        
        # LSTMレイヤの中間変数を初期化
        self.hs = None
    
    # 順伝播メソッド
    def forward(self, xs):
        # 各レイヤの順伝播を計算
        xs = self.embed.forward(xs)
        hs = self.lstm.forward(xs)
        
        # 逆伝播用に隠れ状態を保存
        self.hs = hs
        
        # T-1番目の隠れ状態をDecoderに出力
        return hs[:, -1, :]
    
    # 逆伝播メソッド
    def backward(self, dh):
        # 隠れ状態を初期化
        dhs = np.zeros_like(self.hs)
        dhs[:, -1, :] = dh # Decoderから入力
        
        # 各レイヤの逆伝播を逆順に計算
        dout = self.lstm.backward(dhs)
        dout = self.embed.backward(dout)
        return dout

###### Dncoder

In [ ]:
# デコーダーの定義
class Decoder:
    # 初期化メソッド
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        # 変数の形状に関する値を取得
        V, D, H = vocab_size, wordvec_size, hidden_size
        
        # パラメータを初期化
        embed_W = (np.random.randn(V, D) * 0.01).astype('f')
        lstm_Wx = (np.random.randn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (np.random.randn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (np.random.randn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')
        
        # レイヤを生成
        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
        self.affine = TimeAffine(affine_W, affine_b)
        
        # パラメータと勾配をリストに格納
        self.params = [] # パラメータ
        self.grads = []  # 勾配
        for layer in (self.embed, self.lstm, self.affine):
            self.params += layer.params
            self.grads += layer.grads
        
    # 順伝播メソッド
    def forward(self, xs, h):
        # Encoderの隠れ状態を入力
        self.lstm.set_state(h)
        
        # 各レイヤの順伝播を計算
        # Time Embedレイヤの順伝播を計算
        out = self.embed.forward(xs)
        # Time LSTMレイヤの順伝播を計算
        out = self.lstm.forward(out)
        # Time Affineレイヤの順伝播を計算
        score = self.affine.forward(out)
        return score
    
    # 逆伝播メソッド
    def backward(self, dscore):
        # 各レイヤの逆伝播を逆順に計算
        # Time Affineレイヤの逆伝播を計算
        dout = self.affine.backward(dscore)
        # Time LSTMレイヤの逆伝播を計算
        dout = self.lstm.backward(dout)
        # Time Embedレイヤの逆伝播を計算
        dout = self.embed.backward(dout)
        
        # EncoderのT-1番目の隠れ状態の勾配をEncoderに出力
        dh = self.lstm.dh
        return dh
    
    # 文章生成メソッド
    def generate(self, h, start_id, sample_size):
        # 文字IDの受け皿を初期化
        sampled = []
        
        # 区切り文字のIDを設定
        sample_id = start_id
        
        # エンコードされた足し算の式を入力
        self.lstm.set_state(h)
        
        # 解答を生成
        for _ in range(sample_size):
            # 入力用に2次元配列に変換
            x = np.array(sample_id).reshape((1, 1))
            
            # スコアを計算
            out = self.embed.forward(x)
            out = self.lstm.forward(out)
            score = self.affine.forward(out)
            
            # スコアが最大の文字IDを取得
            sample_id = np.argmax(score.flatten()) # 入力データを更新
            sampled.append(sample_id) # サンプルを保存
        
        return sampled

###### PeekyDecoder

In [ ]:
# Peeky版デコーダーの定義
class PeekyDecoder:
    # 初期化メソッド
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        # 変数の形状に関する値を取得
        V, D, H = vocab_size, wordvec_size, hidden_size
        
        # パラメータを初期化
        embed_W = (np.random.randn(V, D) * 0.01).astype('f')
        #lstm_Wx = (np.random.randn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wx = (np.random.randn(H + D, 4 * H) / np.sqrt(H + D)).astype('f')
        lstm_Wh = (np.random.randn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        #affine_W = (np.random.randn(H, V) / np.sqrt(H)).astype('f')
        affine_W = (np.random.randn(H + H, V) / np.sqrt(H + H)).astype('f')
        affine_b = np.zeros(V).astype('f')
        
        # レイヤを生成
        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
        self.affine = TimeAffine(affine_W, affine_b)
        
        # パラメータと勾配をリストに格納
        self.params = [] # パラメータ
        self.grads = []  # 勾配
        for layer in (self.embed, self.lstm, self.affine):
            self.params += layer.params
            self.grads += layer.grads
        
        # --- 追加 ----------
        # 中間変数の受け皿
        self.cache = None
        # -------------------
        
    # 順伝播メソッド
    def forward(self, xs, h):
        # --- 追加 ----------
        # 変数の形状に関する値を取得
        N, T = xs.shape
        N, H = h.shape
        # -------------------
        
        # Encoderの隠れ状態を入力
        self.lstm.set_state(h)
        
        # 各レイヤの順伝播を計算
        # Time Embedレイヤの順伝播を計算
        out = self.embed.forward(xs)
        # --- 追加 ----------
        # Encoderの隠れ状態を複製
        hs = np.repeat(h, T, axis=0).reshape((N, T, H))
        # Encoderの隠れ状態(複製)と単語ベクトルを結合
        out = np.concatenate((hs, out), axis=2)
        # -------------------
        # Time LSTMレイヤの順伝播を計算
        out = self.lstm.forward(out)
        # --- 追加 ----------
        # Encoderの隠れ状態(複製)とDecoderの隠れ状態を結合
        out = np.concatenate((hs, out), axis=2)
        # -------------------
        # Time Affineレイヤの順伝播を計算
        score = self.affine.forward(out)
        
        # --- 追加 ----------
        # 逆伝播用に変数の形状に関する値を保存
        self.cache = H
        # -------------------
        
        return score
    
    # 逆伝播メソッド
    def backward(self, dscore):
        
        # --- 追加 ----------
        # 変数の形状に関する値を取得
        H = self.cache
        # -------------------
        
        # 各レイヤの逆伝播を逆順に計算
        # Time Affineレイヤの逆伝播を計算
        dout = self.affine.backward(dscore)
        
        # --- 追加 ----------
        # 勾配を分割
        dhs0 = dout[:, :, :H] # Encoderの隠れ状態(複製)の勾配
        dout = dout[:, :, H:] # Decoderの隠れ状態の勾配
        # -------------------
        
        # Time LSTMレイヤの逆伝播を計算
        dout = self.lstm.backward(dout)
        
        # --- 追加 ----------
        # 勾配を分割
        dhs1 = dout[:, :, :H]   # Encoderの隠れ状態(複製)の勾配
        dembed = dout[:, :, H:] # 単語ベクトルの勾配
        # -------------------
        
        # Time Embedレイヤの逆伝播を計算
        dout = self.embed.backward(dembed) # 返り値はNone
        
        # --- 追加 ----------
        # Encoderの隠れ状態の勾配を計算
        dhs = dhs0 + dhs1
        dh = np.sum(dhs, axis=1)
        # -------------------
        
        # EncoderのT-1番目の隠れ状態の勾配を合算してをEncoderに出力
        dh += self.lstm.dh
        return dh
    
    # 文章生成メソッド
    def generate(self, h, start_id, sample_size):
        # 文字IDの受け皿を初期化
        sampled = []
        
        # 区切り文字のIDを設定
        sample_id = start_id
        
        # エンコードされた足し算の式を入力
        self.lstm.set_state(h)
        
        # 解答を生成
        
        # --- 追加 ----------
        H = h.shape[1]
        peeky_h = h.reshape((1, 1, H))
        # -------------------
        
        for _ in range(sample_size):
            # 入力用に2次元配列に変換
            x = np.array(sample_id).reshape((1, 1))
            
            # スコアを計算
            out = self.embed.forward(x)
            # --- 追加 ----------
            out = np.concatenate((peeky_h, out), axis=2)
            # -------------------
            out = self.lstm.forward(out)
            # --- 追加 ----------
            out = np.concatenate((peeky_h, out), axis=2)
            # -------------------
            score = self.affine.forward(out)
            
            # スコアが最大の文字IDを取得
            sample_id = np.argmax(score.flatten()) # 入力データを更新
            sampled.append(sample_id) # サンプルを保存
        
        return sampled

###### Seq2seq

In [ ]:
# seq2seqの実装
class Seq2seq:
    # 初期化メソッド
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        # 変数の形状に関する値を取得
        V, D, H = vocab_size, wordvec_size, hidden_size
        
        # 各レイヤのインスタンスを作成
        self.encoder = Encoder(V, D, H)
        self.decoder = Decoder(V, D, H)
        self.softmax = TimeSoftmaxWithLoss()
        
        # パラメータと勾配をリストに格納
        self.params = self.encoder.params + self.decoder.params # パラメータ
        self.grads = self.encoder.grads + self.decoder.grads    # 勾配
    
    # 順伝播メソッド
    def forward(self, xs, ts):
        # Decoder用のデータを作成
        decoder_xs = ts[:, :-1] # 入力データ:(最後を除く)
        decoder_ts = ts[:, 1:]  # 教師データ:(最初を除く)
        
        # 各レイヤの順伝播を計算
        h = self.encoder.forward(xs)
        score = self.decoder.forward(decoder_xs, h)
        loss = self.softmax.forward(score, decoder_ts)
        return loss
    
    # 逆伝播メソッド
    def backward(self, dout=1):
        # 各レイヤの逆伝播を逆順に計算
        dout = self.softmax.backward(dout)
        dh = self.decoder.backward(dout)
        dout = self.encoder.backward(dh)
        return dout
    
    # 文章生成メソッド
    def generate(self, xs, start_id, sample_size):
        # 足し算の式をエンコード
        h = self.encoder.forward(xs)
        
        # 解答を生成
        sampled = self.decoder.generate(h, start_id, sample_size)
        return sampled

###### PeekySeq2seq

In [ ]:
# seq2seqの実装
class PeekySeq2seq(Seq2seq):
    # 初期化メソッド
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        # 変数の形状に関する値を取得
        V, D, H = vocab_size, wordvec_size, hidden_size
        
        # 各レイヤのインスタンスを作成
        self.encoder = Encoder(V, D, H)
        self.decoder = PeekyDecoder(V, D, H)
        self.softmax = TimeSoftmaxWithLoss()
        
        # パラメータと勾配をリストに格納
        self.params = self.encoder.params + self.decoder.params # パラメータ
        self.grads = self.encoder.grads + self.decoder.grads    # 勾配

##### 学習の実行

In [ ]:
# 文字の種類数(EmbedレイヤとAffineレイヤのニューロン数)を取得
vocab_size = len(char_to_id)

# 単語ベクトルのサイズ(Embedレイヤのニューロン数)を指定
wordvec_size = 16

# 隠れ状態のサイズ(LSTMレイヤのニューロン数)を指定
hidden_size = 256

# バッチサイズを指定
batch_size = 128

# エポック当たりの試行回数を指定
max_epoch = 10

# 勾配の閾値を指定
max_grad = 5.0

# Attention seq2seqのインスタンスを作成
attention_model = _AttentionSeq2seq(vocab_size, wordvec_size, hidden_size)
attention_optimizer = Adam()
attention_trainer = Trainer(attention_model, attention_optimizer)

# Peeky seq2seqのインスタンスを作成
peeky_model = PeekySeq2seq(vocab_size, wordvec_size, hidden_size)
peeky_optimizer = Adam()
peeky_trainer = Trainer(peeky_model, peeky_optimizer)

# Attention seq2seqのインスタンスを作成
base_model = Seq2seq(vocab_size, wordvec_size, hidden_size)
base_optimizer = Adam()
base_trainer = Trainer(base_model, base_optimizer)

# 正解率の記録用のリストを初期化
attention_acc_list = []
peeky_acc_list = []
base_acc_list = []

# 繰り返し試行
for epoch in range(max_epoch):
    # 学習
    print('----- Attention seq2seq -----')
    attention_trainer.fit(reverse_x_train, t_train, max_epoch=1, batch_size=batch_size, max_grad=max_grad)
    print('----- Peeky seq2seq -----')
    peeky_trainer.fit(reverse_x_train, t_train, max_epoch=1, batch_size=batch_size, max_grad=max_grad)
    print('----- seq2seq -----')
    base_trainer.fit(reverse_x_train, t_train, max_epoch=1, batch_size=batch_size, max_grad=max_grad)
    
    # 正解数を初期化
    attention_correct_num = 0
    peeky_correct_num = 0
    base_correct_num = 0
    
    # 精度を測定
    for n in range(len(reverse_x_test)):
        # データを取得
        question = reverse_x_test[[n]]  # Encoderの入力データ(足し算の式)
        start_id = t_test[n, 0] # Decoderの入力データの最初の文字(区切り文字)
        correct = t_test[n, 1:] # 教師データ(足し算の答)
        
        # 解答を生成
        attention_guess = attention_model.generate(question, start_id, len(correct))
        peeky_guess = peeky_model.generate(question, start_id, len(correct))
        base_guess = base_model.generate(question, start_id, len(correct))
        
        # 正解数をカウント
        if attention_guess == list(correct): # 解答と正答が一致したら
            attention_correct_num += 1
        if peeky_guess == list(correct): # 解答と正答が一致したら
            peeky_correct_num += 1
        if base_guess == list(correct): # 解答と正答が一致したら
            base_correct_num += 1
    
    # 正解率を計算
    attention_acc = float(attention_correct_num) / len(reverse_x_test)
    attention_acc_list.append(attention_acc)
    peeky_acc = float(peeky_correct_num) / len(reverse_x_test)
    peeky_acc_list.append(peeky_acc)
    base_acc = float(base_correct_num) / len(reverse_x_test)
    base_acc_list.append(base_acc)
    
    # 途中経過を表示
    print('----- results -----')
    print('val attention acc:' + str(attention_acc * 100))
    print('val peeky acc:' + str(peeky_acc * 100))
    print('val base acc:' + str(base_acc * 100))

###### 学習結果を表示する。

In [ ]:
# 作図
plt.figure(figsize=(9, 6))
plt.plot(1 + np.arange(len(base_acc_list)), base_acc_list, 
         marker='v', label='baseline') # seq2seqの結果
plt.plot(1 + np.arange(len(peeky_acc_list)), peeky_acc_list, 
         marker='D', label='peeky') # Peeky seq2seqの結果
plt.plot(1 + np.arange(len(attention_acc_list)), attention_acc_list, 
         marker='o', label='attention') # Attention seq2seqの結果
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('seq2seq', fontsize=20)
#plt.ylim(0, 1) # y軸の表示範囲
plt.grid() # グリッド線

###### 学習結果をロードする。

In [ ]:
# 保存と復元
import pickle

# 保存されるオブジェクト
#obj = [base_acc_list, peeky_acc_list, attention_acc_list]

# シリアライズ相当
#with open('../work/NekozameRNN4_acc_list.pickle','wb') as f:
#   pickle.dump(obj, f)

# デシリアライズ相当
with open('../work/NekozameRNN4_acc_list.pickle','rb') as f:
   loaded_obj = pickle.load(f)

base_acc_list, peeky_acc_list, attention_acc_list = loaded_obj